In [1]:
import os
import sys
import tqdm
import pandas as pd

In [2]:
REBADD_LIB_PATH = os.path.abspath(os.pardir)
if REBADD_LIB_PATH not in sys.path:
    sys.path = [REBADD_LIB_PATH] + sys.path

from rebadd.evaluate import evaluate_sr_nov_div

In [3]:
## Test data (navitoclax, ABT-737)
filepath_ref = os.path.join(os.pardir, 'data', 'zinc15', 'zinc15_test.txt')
df_ref = pd.read_csv(filepath_ref, header=None)

referece_smiles_iter = df_ref.iloc[:,0].values.tolist()

print(len(referece_smiles_iter))

2


In [4]:
input_dir = 'outputs_7_calculate_properties_generated'

modelnames = [
    'zinc15',
]

filenames = [f'smi_after.csv.{num}' for num in range(10)]

In [5]:
frames = []

for modelname in modelnames:
    
    for filename in filenames:
        
        filepath = os.path.join(input_dir, modelname, filename)
        
        df = pd.read_csv(filepath)
        
        df = df.loc[:,('smiles', 'bcl2', 'bclxl', 'bclw')]
        df.loc[:,'model'] = modelname
        df.loc[:,'checkpoint'] = filename.split('.')[-1]
        
        frames.append(df)

In [6]:
data = []

for df in tqdm.tqdm(frames):

    s_sr, s_nov, s_div = evaluate_sr_nov_div(df, referece_smiles_iter, 'bcl2_bclxl_bclw')
    
    data.append({'SR':s_sr, 'Nov':s_nov, 'Div':s_div, 'Model':df.loc[0,'model'], 'Ckpt':df.loc[0,'checkpoint']})

100%|██████████| 10/10 [00:09<00:00,  1.11it/s]


In [7]:
df_records = pd.DataFrame(data)
df_records.loc[:,'HMean'] = (df_records.loc[:,'SR'] * df_records.loc[:,'Nov'] * df_records.loc[:,'Div']) ** 0.333

df_records

,SR,Nov,Div,Model,Ckpt,HMean
0,0.4626,1.0,0.646724,zinc15,0,0.669087
1,0.4740,1.0,0.623341,zinc15,1,0.666312
2,0.4746,1.0,0.660504,zinc15,2,0.679572
3,0.4710,1.0,0.629092,zinc15,3,0.666942
4,0.4788,1.0,0.652321,zinc15,4,0.678746
5,0.4736,1.0,0.635146,zinc15,5,0.670300
6,0.4738,1.0,0.656506,zinc15,6,0.677819
7,0.4622,1.0,0.633079,zinc15,7,0.664162
8,0.4766,1.0,0.646409,zinc15,8,0.675654
9,0.4702,1.0,0.642957,zinc15,9,0.671421


In [8]:
df_records.groupby('Model').mean(numeric_only=True)

,SR,Nov,Div,HMean
Model,,,,
zinc15,0.47174,1.0,0.642608,0.672001


In [9]:
df_records.groupby('Model').std(numeric_only=True)

,SR,Nov,Div,HMean
Model,,,,
zinc15,0.005497,0.0,0.0122,0.005589
